In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv('Diff-stock.csv')

Here I am impporting the variables which currently I am working on. Here I am trying to find out that for example, this is a portfolio of Govt which includes share market, Fed Balance Sheet, Federal Funds rate and the Loss Function and I amhere trying to find out that on which variable Govt. should give more focus as well as more weight in the time of taking decision.

Here, the datasets are twice differenced to ensure stationarity.

In [3]:
df

,DATE,S&P 500,TA,CC,FFR,LF
0,2010-03,0.029018,0.008611,-0.000738,0.021659,-1.133584
1,2010-04,-0.042481,-0.000267,-0.003030,-0.004270,1.019559
2,2010-05,-0.100183,-0.009751,-0.000112,-0.031505,3.585353
3,2010-06,0.030143,-0.003164,0.001554,-0.026290,-1.340778
4,2010-07,0.121904,-0.002388,-0.000532,0.029892,-2.219644
...,...,...,...,...,...,...
113,2019-08,-0.031300,0.001927,-0.003976,-0.302333,0.109370
114,2019-09,0.035198,0.017413,0.006447,0.194290,0.050563
115,2019-10,0.003379,0.032283,-0.003852,-0.130516,-0.187102
116,2019-11,0.013160,-0.023511,0.005203,-0.063022,-0.018272


Here,
 'S&P 500' = S & P 500 index
 'TA' =  Total Asset
 'CC' = Currency in Circulation
 'FFR'= Federal Funds Rate
 'LF'= Loss Function

In [4]:
df = df.set_index('DATE')
df

,S&P 500,TA,CC,FFR,LF
DATE,,,,,
2010-03,0.029018,0.008611,-0.000738,0.021659,-1.133584
2010-04,-0.042481,-0.000267,-0.003030,-0.004270,1.019559
2010-05,-0.100183,-0.009751,-0.000112,-0.031505,3.585353
2010-06,0.030143,-0.003164,0.001554,-0.026290,-1.340778
2010-07,0.121904,-0.002388,-0.000532,0.029892,-2.219644
...,...,...,...,...,...
2019-08,-0.031300,0.001927,-0.003976,-0.302333,0.109370
2019-09,0.035198,0.017413,0.006447,0.194290,0.050563
2019-10,0.003379,0.032283,-0.003852,-0.130516,-0.187102


The below I am going to calculate the mean of the variables:

In [5]:
returns = df.pct_change()

In [6]:
returns

,S&P 500,TA,CC,FFR,LF
DATE,,,,,
2010-03,NaN,NaN,NaN,NaN,NaN
2010-04,-2.463963,-1.031030,3.106258,-1.197163,-1.899412
2010-05,1.358288,35.494180,-0.962909,6.377697,2.516571
2010-06,-1.300882,-0.675489,-14.829711,-0.165529,-1.373960
2010-07,3.044158,-0.245472,-1.342033,-2.137014,0.655489
...,...,...,...,...,...
2019-08,-0.416213,-1.819313,-2.935127,-8.845145,-1.842807
2019-09,-2.124549,8.034076,-2.621516,-1.642636,-0.537688
2019-10,-0.904010,0.854013,-1.597503,-1.671758,-4.700371


In [16]:
returns = returns.dropna()
returns

,S&P 500,TA,CC,FFR,LF
DATE,,,,,
2010-04,-2.463963,-1.031030,3.106258,-1.197163,-1.899412
2010-05,1.358288,35.494180,-0.962909,6.377697,2.516571
2010-06,-1.300882,-0.675489,-14.829711,-0.165529,-1.373960
2010-07,3.044158,-0.245472,-1.342033,-2.137014,0.655489
2010-08,-1.944411,0.899624,3.154707,-0.971223,-0.520509
...,...,...,...,...,...
2019-08,-0.416213,-1.819313,-2.935127,-8.845145,-1.842807
2019-09,-2.124549,8.034076,-2.621516,-1.642636,-0.537688
2019-10,-0.904010,0.854013,-1.597503,-1.671758,-4.700371


In [17]:
returns.to_csv("Returns.csv")

In [8]:
returns.mean()

S&P 500    8.081081
TA         1.050390
CC        -9.546617
FFR       -0.834054
LF         6.756371
dtype: float64

In [9]:
returns.mean().mean()

1.101434128898222

Now, I will calculate the weight of the variable with the below automated code:

In [10]:
# the objective function is to minimize the portfolio risk
def objective(weights): 
    weights = np.array(weights)
    return weights.dot(returns.cov()).dot(weights.T)
# The constraints
cons = (# The weights must sum up to one.
        {"type":"eq", "fun": lambda x: np.sum(x)-1}, 
        # This constraints says that the inequalities (ineq) must be non-negative.
        # The expected daily return of our portfolio and we want to be at greater than 0.002352
        {"type": "ineq", "fun": lambda x: np.sum(returns.mean()*x)-1.2})
# Every stock can get any weight from 0 to 1
bounds = tuple((0,1) for x in range(returns.shape[1]))
# Initialize the weights with an even split
# In out case each stock will have 10% at the beginning
guess = [1./returns.shape[1] for x in range(returns.shape[1])]
optimized_results = minimize(objective, guess, method = "SLSQP", bounds=bounds, constraints=cons)
optimized_results

     fun: 167.70641659804164
     jac: array([1283.26727104,  314.80454636,   -6.95209694,   55.22638321,
       1100.79135323])
 message: 'Optimization terminated successfully'
    nfev: 46
     nit: 7
    njev: 7
  status: 0
 success: True
       x: array([0.05394617, 0.42353328, 0.        , 0.42305411, 0.0994665 ])

The optimum weights are the array x and we can retrieve them as follows:

In [11]:
optimized_results.x

array([0.05394617, 0.42353328, 0.        , 0.42305411, 0.0994665 ])

We can check that the weights sum up to 1:

In [12]:
# we get 1
np.sum(optimized_results.x)

1.0000000594068226

We can see that the expected return of the portfolio is:

In [13]:
np.sum(returns.mean()*optimized_results.x)

1.2000007449567138

Which is almost 0.00008044 (some rounding errors) which was our requirement.

In [14]:
symbols = ['S&P 500', 'TA', 'CC', 'FFR', 'LF']

In [15]:
pd.DataFrame(list(zip(symbols, optimized_results.x)), 
                       columns=['Symbol', 'Weight'])

,Symbol,Weight
0,S&P 500,0.053946
1,TA,0.423533
2,CC,0.000000
3,FFR,0.423054
4,LF,0.099467


From the above results, we can say that Govt should give more focus on TA, FFR, LF, S&P 500 index.

After that, I can do another model for example, I can run regression analysis or anything on this project if you like it. Moreover, I also can add other variables as well like the yield curve.